In [1]:
import time
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

def accept_cookies():
    try:
        accept_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Accept') or contains(text(), 'Agree')]")
        accept_button.click()
    except:
        pass

def extract_details():
    articles = driver.find_elements(By.CSS_SELECTOR, "article.collection__item")
    details = []

    for article in articles:
        try:
            date_text = article.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")
            date_object = datetime.strptime(date_text, "%d/%m/%y")
        except:
            date_object = None

        try:
            title = article.find_element(By.CSS_SELECTOR, "h4.collection__item-heading span").text
        except:
            title = ""

        try:
            description = article.find_element(By.CSS_SELECTOR, "p.paragraph").text
        except:
            description = ""

        try:
            link = article.find_element(By.CSS_SELECTOR, "a.collection__item-link").get_attribute("href")
        except:
            link = ""

        details.append({
            "Date": date_object,
            "Title": title,
            "Description": description,
            "Link": link
        })

    return details

def click_load_more():
    try:
        load_more_button = driver.find_element(By.CSS_SELECTOR, "button.collection__load-more")
        if "disabled" not in load_more_button.get_attribute("class"):
            load_more_button.click()
            time.sleep(2)
            return True
    except:
        pass
    return False

end_date = datetime.now()
start_date = end_date - timedelta(days=30)

driver.get("https://www.pwc.com/gx/en/research-insights.html")
accept_cookies()
time.sleep(2)

details = extract_details()

while True:
    more_results = click_load_more()
    if not more_results:
        break

    new_details = extract_details()
    if not new_details:
        break

    for detail in new_details:
        if detail["Date"] and start_date <= detail["Date"] <= end_date:
            details.append(detail)
        elif detail["Date"] and detail["Date"] < start_date:
            break

with open("pwc_global_insights_details.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["Date", "Title", "Description", "Link"])
    writer.writeheader()
    for detail in details:
        writer.writerow({
            "Date": detail["Date"].strftime("%d/%m/%y"),
            "Title": detail["Title"],
            "Description": detail["Description"],
            "Link": detail["Link"]
        })

driver.quit()

print(f"Extracted details from {len(details)} articles and saved to pwc_insights_details.csv")


AttributeError: 'NoneType' object has no attribute 'strftime'